# 🧠 Indian Sign Language (ISL) A-Z Classifier
Training notebook for classifying ISL hand gestures A-Z using a CNN with PyTorch.

In [ ]:

# 📦 Imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training on {device}')


In [ ]:

# 🧹 Paths & Hyperparameters
data_dir = '/kaggle/input/indian-sign-language'  # update if needed
batch_size = 64
lr = 0.001
epochs = 30
img_size = 128

# 📈 Transforms
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 🗂️ Dataset
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Dataset'), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
class_names = train_dataset.classes
num_classes = len(class_names)


In [ ]:

# 🧠 Model
class ISLCNN(nn.Module):
    def __init__(self, num_classes):
        super(ISLCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (img_size//8) * (img_size//8), 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = ISLCNN(num_classes).to(device)


In [ ]:

# ⚙️ Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [ ]:

# 🔁 Training Loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss:.3f} | Accuracy: {acc:.2f}%")


In [ ]:

# 💾 Save Model
torch.save(model.state_dict(), 'isl_cnn_model.pth')


In [ ]:

# 📤 Export to ONNX
dummy_input = torch.randn(1, 3, img_size, img_size, device=device)
torch.onnx.export(model, dummy_input, 'isl_cnn_model.onnx', input_names=['input'], output_names=['output'], opset_version=11)
print("ONNX model exported.")
